# Sporulation

In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP
import os

# python imports
from copy import copy
import re
from os.path import join, dirname, abspath
import sys
from collections import defaultdict
import pickle

# third party imports
import pandas as pd
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from cobrame.util.helper_functions import *

import copy
from scipy import stats
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
ecoli_files = dirname(abspath(bacillusme.__file__))
pd.set_option('display.max_colwidth', None)

/home/jt/work/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/work/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/work/cobrapy-0.5.11/cobra/__init__.py
/home/jt/work/cobrame/cobrame/__init__.py
/home/jt/work/sporeme/bacillusme/__init__.py


In [2]:
def add_transport_to_model(model,mets):
    for met in mets:
        rxn = cobrame.MEReaction('{}_transport'.format(met))
        model.add_reactions([rxn])
        rxn.add_metabolites({
            '{}_c'.format(met):-1,
            '{}_s'.format(met):1
        })
        rxn.bounds = (-1000,1000)
        print(rxn.reaction)

### Essentiality of FS genes

In [3]:
with open("./sporeme_solution_v4_proteomics_spoAQ_naive.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [4]:
add_transport_to_model(sporeme,['gtp','ctp','utp','gdp','cdp','udp','adp','atp'])

gtp_c <=> gtp_s
ctp_c <=> ctp_s
utp_c <=> utp_s
gdp_c <=> gdp_s
cdp_c <=> cdp_s
udp_c <=> udp_s
adp_c <=> adp_s
atp_c <=> atp_s


In [ ]:
from bacillusme.analysis import sensitivity as ss

In [6]:
genes = ['BSU' + g.id.split('RNA_BSU')[1] for g in sporeme.metabolites.query(re.compile('RNA_BSU.*_s$'))]

In [7]:
solve_me_model(sporeme,
                   max_mu = 0.1,
                   min_mu = .01,
                   using_soplex=False,
                   precision = 1e-6,
                   growth_key = 'sigma',
                   verbosity=2)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 53.648077 seconds
Finished substituting S,lb,ub in 1.850711 seconds
Finished makeME_LP in 0.262818 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.05 0.05 1
Finished substituting S,lb,ub in 2.122538 seconds
Finished makeME_LP in 0.312662 seconds
Getting MINOS parameters from ME_NLP...
2 0.025 0.025 0.05 0.025 optimal
Finished substituting S,lb,ub in 2.100982 seconds
Finished makeME_LP in 0.267643 seconds
Getting MINOS parameters from ME_NLP...
3 0.025 0.025 0.037500000000000006 0.037500000000000006 1
Finished substituting S,lb,ub in 2.071897 seconds
Finished makeME_LP in 0.267238 seconds
Getting MINOS parameters from ME_NLP...
4 0.03125 0.03125 0.037500000000000006 0.03125 optimal
Finished substituting S,lb,ub in 2.119460 seconds
Finished makeME_LP in 0.271038 seconds
Getting MINOS parameters from ME_NLP...
5 0.034375 0.034375 0.037500000000000006 0.034375 optimal
Finished substituting S,lb,ub i

In [ ]:
for i in range(0,int(len(genes)/100)+1):
    a,b = (i*100,i*100 + 100)
    b = min(b,len(genes))
    
    print(i+1,a,b)
    genes_it = genes[a:b]
    flux_responses = ss.all_flux_responses(sporeme,genes_it,NP=10,precision=1e-6,max_mu=0.2,
                                           solution=sporeme.solution,
                                           single_change_function='gene_knockout',growth_key='sigma')
    
    missing = list(set(genes_it) - set(flux_responses.columns))
    flux_responses[missing] = [[np.nan]*len(missing)]*flux_responses.shape[0]
    flux_responses.to_csv('./essentiality_raw_results_naive_FS_transport_spoAQ_{}.csv'.format(i+1))

1 0 100


dummy_reaction_FWD_SPONT_s response (10 threads):  31%|███       | 31/100 [1:22:27<2:52:33, 150.04s/it]

In [ ]:
flux_responses = {}
f = 'essentiality_raw_results_naive_FS_transport_spoAQ'
for i in tqdm(os.listdir()):
    if f not in i: continue
    if f+'.csv' == i: continue
    d = pd.read_csv(i,index_col=0).dropna().to_dict()
    for k,v in d.items():
        flux_responses[k] = v
flux_responses = pd.DataFrame.from_dict(flux_responses)
flux_responses.to_csv('{}.csv'.format(f))

In [ ]:
df = flux_responses.T
repeat = []
for i,row in df.drop('base').iterrows():
    f = row['translation_'+i]
    if  f>0.:
        repeat.append(i)
repeat

In [ ]:
del flux_responses

In [ ]:
2